In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl wandb python-dotenv
!pip install -qqq flash-attn
!pip list |egrep 'accelerate|peft|bitsandbytes|transformers|trl|wandb'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
accelerate                         0.34.2
bitsandbytes                       0.44.1
fastrlock                          0.8.2
peft                               0.13.2
transformers                       4.44.2
trl                      

In [2]:
import huggingface_hub
import os
load_dotenv()
access_token_read = os.getenv("HUGGINGFACE_TOKEN")
huggingface_hub.login(token = access_token_read)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import wandb
wandb_token=os.getenv("WANDB")
wandb.login(key=wandb_token)
run = wandb.init(
    project='Fine-tune Llama 3.2 on ai-medical-chatbot and PubMedQA Datasets',
    job_type="training",
    anonymous="allow",
    resume="allow"
    )

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: youssefkardous1 (youssefkardous1-personal). Use `wandb login --relogin` to force relogin


In [4]:
!git lfs install
!git clone https://huggingface.co/youzarsif/Doctor_Llama-3.2-3B-Instruct

Git LFS initialized.
Cloning into 'Doctor_Llama-3.2-3B-Instruct'...
remote: Enumerating objects: 347, done.
remote: Counting objects: 100% (344/344), done.
remote: Compressing objects: 100% (344/344), done.
remote: Total 347 (delta 100), reused 0 (delta 0), pack-reused 3 (from 1)
Receiving objects: 100% (347/347), 2.30 MiB | 2.72 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Filtering content: 100% (8/8), 3.20 GiB | 41.34 MiB/s, done.


In [5]:
import os, torch
from trl import SFTTrainer, setup_chat_format
from peft import LoraConfig,get_peft_model
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    pipeline,
)
import bitsandbytes as bnb


In [6]:
model_name="meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
from pprint import pprint
from datasets import load_dataset
import pandas as pd
import re

ai_med_chat = load_dataset("ruslanmv/ai-medical-chatbot", split="train[:9%]")
pub_med = load_dataset("qiaojin/PubMedQA","pqa_artificial", split="train[:11%]")

ai_med_chat = ai_med_chat.filter(lambda x: x["Description"] != "" or x["Patient"]!= "" or x["Doctor"]!= "" )
pub_med = pub_med.filter(lambda x: x["question"] != "" or x["long_answer"]!= "")

pd.DataFrame(ai_med_chat)
pd.DataFrame(pub_med)

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23122 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23240 [00:00<?, ? examples/s]

,pubid,question,context,long_answer,final_decision
0,25429730,Are group 2 innate lymphoid cells ( ILC2s ) in...,{'contexts': ['Chronic rhinosinusitis (CRS) is...,"As ILC2s are elevated in patients with CRSwNP,...",yes
1,25433161,Does vagus nerve contribute to the development...,{'contexts': ['Phosphatidylethanolamine N-meth...,Neuronal signals via the hepatic vagus nerve c...,yes
2,25445714,Does psammaplin A induce Sirtuin 1-dependent a...,{'contexts': ['Psammaplin A (PsA) is a natural...,PsA significantly inhibited MCF-7/adr cells pr...,yes
3,25431941,Is methylation of the FGFR2 gene associated wi...,{'contexts': ['This study examined links betwe...,We identified a novel biologically plausible c...,yes
4,25432519,Do tumor-infiltrating immune cell profiles and...,{'contexts': ['Tumor microenvironment immunity...,Breast cancer immune cell subpopulation profil...,yes
...,...,...,...,...,...
23235,11004348,Do wet smear compared with gram stain diagnosi...,{'contexts': ['To compare wet smear and Gram s...,"In asymptomatic pregnant women, bacterial vagi...",yes
23236,11017940,Does the haptoglobin 2-2 phenotype affect seru...,{'contexts': ['Human iron status is influenced...,The Hp 2-2 phenotype affects serum iron status...,yes
23237,11029150,Does caffeine decrease the occurrence of cadmi...,{'contexts': ['Cadmium is a well-known animal ...,This study provides evidence that a subteratog...,yes
23238,11029153,Is hydroxylamine moiety of developmental toxic...,"{'contexts': ['Cellular debris, an indicator o...",These results are consistent with the concept ...,yes


In [8]:

instruct_ai_med_chat = """
You are an AI doctor trained to provide clear, compassionate, and accurate health information.
Your goal is to answer medical questions, suggest potential treatments, and offer general health advice
based on established medical guidelines.
"""

def clean_description(description):
    cleaned = re.sub(r'^Q\.\s*', '', description.strip())
    cleaned = re.sub(r'\s+Q\.\s*', ' ', cleaned)
    return cleaned

def format_chat_template1(row):

    cleaned_description = clean_description(row["Description"])

    row_json = [{"role": "system", "content": instruct_ai_med_chat },
               {"role": "user", "content": row["Patient"] + " " + cleaned_description},
               {"role": "assistant", "content": row["Doctor"]  }]

    row["labeled_data"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

ai_med_chat = ai_med_chat.map(format_chat_template1,
                                        num_proc= 4
                                    )

instruct_pub_med="""
You are an AI trained to interpret and analyze scientific research papers.Your task is to extract key findings,
summarize clinical studies, and provide evidence-based insights to support medical advice.
"""

def format_chat_template2(row):

    row_json = [{"role": "system", "content": instruct_pub_med },
               {"role": "user", "content": row["question"] },
               {"role": "assistant", "content": row["long_answer"]  }]

    row["labeled_data"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

pub_med = pub_med.map(format_chat_template2,
                                        num_proc= 4
                                    )

print("Number of examples: ", len(pub_med),"   ",len(ai_med_chat))

Map (num_proc=4):   0%|          | 0/23122 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/23240 [00:00<?, ? examples/s]

Number of examples:  23240     23122


In [9]:
pprint(ai_med_chat[30]["labeled_data"])
pprint(pub_med[30]["labeled_data"])

('<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n'
 '\n'
 'Cutting Knowledge Date: December 2023\n'
 'Today Date: 26 Jul 2024\n'
 '\n'
 'You are an AI doctor trained to provide clear, compassionate, and accurate '
 'health information.\n'
 'Your goal is to answer medical questions, suggest potential treatments, and '
 'offer general health advice\n'
 'based on established medical '
 'guidelines.<|eot_id|><|start_header_id|>user<|end_header_id|>\n'
 '\n'
 'Dear Doctor, I am trying to figure out what is wrong with my 15 year old '
 'daughter. Her symptoms have developed most significantly over the past few '
 'months, and include depression, weakness, constant light-headedness, '
 'shortness of breath (even when sitting still), difficulty taking a deep '
 'breath (causes pain), episodes where breathing becomes especially difficult, '
 'a complete lack of energy, continual exhaustion, continual muscle soreness '
 '(especially in the calves), cold hands and feet (to the point

In [10]:
def split_to_train_test_eval(dataset):
    dataset = dataset.train_test_split(test_size=0.2, seed=42)
    train= dataset['train']
    dataset = dataset['test'].train_test_split(test_size=0.5, seed=42)
    eval= dataset['train']
    test= dataset['test']
    return train, eval, test

In [11]:
train_ai_med_chat, eval_ai_med_chat, test_ai_med_chat = split_to_train_test_eval(ai_med_chat)
train_pub_med, eval_pub_med, test_pub_med = split_to_train_test_eval(pub_med)

In [12]:
def move_2_df(train, eval, test):
    df__train = pd.DataFrame(train)
    df__eval = pd.DataFrame(eval)
    df__test = pd.DataFrame(test)
    return df__train, df__eval, df__test

In [13]:
df__train_ai_med_chat, df__eval_ai_med_chat, df__test_ai_med_chat = move_2_df(train_ai_med_chat, eval_ai_med_chat, test_ai_med_chat)
df__train_pub_med, df__eval_pub_med, df__test_pub_med = move_2_df(train_pub_med, eval_pub_med, test_pub_med)

In [14]:
df__train_ai_med_chat.head(3)


,Description,Patient,Doctor,labeled_data
0,Q. Please explain the poisonous effect of phen...,"Hello doctor, My friend aged 30 had two drops ...",Hi. I want to assure you not to worry as every...,<|begin_of_text|><|start_header_id|>system<|en...
1,Q. Will Kalarchikai cure multiple ovarian cyst...,"Hello doctor, I have multiple small cysts in b...",Hello. I just read your query. See Kalarachi K...,<|begin_of_text|><|start_header_id|>system<|en...
2,Q. Why could be the reason for breathing diffi...,"Hello doctor,My mother is 59 years old and had...",Hello. I have gone through all the history you...,<|begin_of_text|><|start_header_id|>system<|en...


In [15]:
df__train_pub_med.head(3)

,pubid,question,context,long_answer,final_decision,labeled_data
0,19545423,Do gold nanoparticles induce cytotoxicity in t...,{'contexts': ['During the last years engineere...,Our data suggest that the presence of contamin...,yes,<|begin_of_text|><|start_header_id|>system<|en...
1,19563391,Are fitness and abdominal obesity independentl...,{'contexts': ['To examine the relationship bet...,Higher fitness and lower waist circumference a...,yes,<|begin_of_text|><|start_header_id|>system<|en...
2,12609097,Is familial Wolff-Parkinson-White syndrome lin...,{'contexts': ['Wolff-Parkinson-White syndrome ...,The gene of Wolff-Parkinson-White syndrome is ...,yes,<|begin_of_text|><|start_header_id|>system<|en...


In [16]:
columns_to_drop1 = ['Description','Patient','Doctor']
columns_to_drop2 = ['question','long_answer','pubid','final_decision','context']
df__train_ai_med_chat = df__train_ai_med_chat.drop(columns=columns_to_drop1)
df__train_pub_med = df__train_pub_med.drop(columns=columns_to_drop2)

In [17]:
def merge_dataframes(df1, df2):
    merged_df = pd.concat([df1, df2], ignore_index=True).sample(frac=1.0)
    return merged_df

In [18]:
df__train= merge_dataframes(df__train_ai_med_chat, df__train_pub_med)
df__eval= merge_dataframes(df__eval_ai_med_chat, df__eval_pub_med)
df__test= merge_dataframes(df__test_ai_med_chat, df__test_pub_med)

In [19]:
df__train.head(3)
#df__eval.head(3)
#df__test.head(3)

,labeled_data
21793,<|begin_of_text|><|start_header_id|>system<|en...
36904,<|begin_of_text|><|start_header_id|>system<|en...
18257,<|begin_of_text|><|start_header_id|>system<|en...


In [20]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(df__train)
eval_dataset=Dataset.from_pandas(df__eval)
test_dataset=Dataset.from_pandas(df__test)

In [21]:
print(train_dataset[18]["labeled_data"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an AI trained to interpret and analyze scientific research papers.Your task is to extract key findings,
summarize clinical studies, and provide evidence-based insights to support medical advice.<|eot_id|><|start_header_id|>user<|end_header_id|>

Does differentiation of monocytes into macrophages induce the upregulation of histamine H1 receptor?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Differentiation of monocytes into macrophages or dendritic cells is associated with profound changes of histamine receptor expression. Upregulation of H(1) receptors confers on macrophages the capacity of being activated by histamine.<|eot_id|>


In [22]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [23]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [24]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
    attn_implementation = "flash_attention_2",
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [25]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()


In [26]:

training_arguments = TrainingArguments(
    output_dir="Doctor_Llama-3.2-3B-Instruct",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    lr_scheduler_type = "cosine",
    warmup_ratio = 0.03,
    group_by_length = True,
    max_grad_norm = 0.3,
    gradient_checkpointing = True,
    push_to_hub=True,
    hub_strategy="checkpoint",
    save_steps=2500,

)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="labeled_data",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train(resume_from_checkpoint="./Doctor_Llama-3.2-3B-Instruct/last-checkpoint")
#trainer.train()
wandb.finish()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `ev

Map:   0%|          | 0/37089 [00:00<?, ? examples/s]

Map:   0%|          | 0/4636 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Epoch,Training Loss,Validation Loss
3,1.059700,1.097110


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁██
train/global_step,▁██
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
eval/loss,1.09711
eval/runtime,631.1051
